In [1]:
import os
base_directory = '/workspaces/streaming-prj'
os.chdir(base_directory)

In [2]:
import socket

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

while True:
    data, addr = sock.recvfrom(1024)  # Buffer size 1024 bytes
    print(f"Received message: {data.decode()}")


Listening on 127.0.0.1:5005
Received message: Hello from Notebook 1


KeyboardInterrupt: 

In [3]:
import socket
import json

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Variable to hold the full message
received_data = []

while True:
    data, addr = sock.recvfrom(1024)  # Buffer size 1024 bytes
    if not data:
        break  # Exit on empty data (end of transmission)
    
    received_data.append(data.decode())  # Store received chunk
    print(f"Received chunk: {data.decode()[:50]}...")  # Show first 50 characters

# Reconstruct the full message
full_data = ''.join(received_data)

# Decode JSON data
try:
    json_message = json.loads(full_data)
    print(f"Received full message: {json_message['message'][:100]}...")  # Show first 100 characters
except json.JSONDecodeError:
    print("Failed to decode JSON data")

sock.close()


Listening on 127.0.0.1:5005
Received chunk: {"message": "This is a much larger message. This i...
Received chunk: s is a much larger message. This is a much larger ...
Received chunk: er message. This is a much larger message. This is...
Received chunk:  is a much larger message. "}...


KeyboardInterrupt: 

In [5]:
import socket

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

print(f"Listening on {UDP_IP}:{UDP_PORT}")

# Variable to hold the binary data
received_data = []
output_file_path = "image/received_image.jpg"  # Path to save the received file

while True:
    data, addr = sock.recvfrom(1024)  # Buffer size 1024 bytes
    if data == b'END':  # Check for the termination signal
        print("Received termination signal: 'END'. Stopping the transfer.")
        break  # Exit the loop when 'END' is received
    
    received_data.append(data)  # Store received chunk
    print(f"Received chunk of size {len(data)} bytes")

# Reconstruct the full binary file
with open(output_file_path, "wb") as output_file:
    for chunk in received_data:
        output_file.write(chunk)

print(f"File received and saved to {output_file_path}")
sock.close()


Listening on 127.0.0.1:5005
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of size 1024 bytes
Received chunk of si

In [ ]:
import socket
import pyaudio

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to listen on

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket
sock.bind((UDP_IP, UDP_PORT))  # Bind to address and port

# Setup PyAudio for audio playback
p = pyaudio.PyAudio()

# Audio settings
FORMAT = pyaudio.paInt16  # Audio format (16-bit PCM)
CHANNELS = 1  # Mono
RATE = 44100  # Sample rate (Hz)
CHUNK = 1024  # Number of frames per buffer

# Open the stream for playback
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True, frames_per_buffer=CHUNK)

print("Receiving and playing audio...")

while True:
    # Receive the audio data over UDP
    audio_data, addr = sock.recvfrom(CHUNK)
    
    if audio_data == b'END':  # Check for the termination signal
        print("Received termination signal: 'END'. Stopping the transfer.")
        break  # Stop receiving when 'END' is received

    # Play the received audio data
    stream.write(audio_data)

# Clean up
sock.close()
stream.stop_stream()
stream.close()
p.terminate()
